In [3]:
#ensembling
from mlxtend.classifier import StackingCVClassifier
import numpy as np
import pandas as pd
import sys, os, glob
import imp
import seaborn as sns
sys.path.append('./src/')
import common_utils,my_ensembler, feature_utils, defines, model_utils, my_bert
from sklearn.neighbors import KNeighborsClassifier
from sklearn_crfsuite import scorers, CRF
from sklearn.pipeline import Pipeline
import json
from operator import itemgetter
from sklearn_crfsuite.utils import flatten
from sklearn_crfsuite.metrics import sequence_accuracy_score
from sklearn.linear_model import LogisticRegression
import scipy
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import classes
import pickle

In [26]:
dir_name='reparse'
crf_pred_name='cv_crf_best_params.78'

In [27]:
imp.reload(classes)
winDiff=classes.WindowDiff()

WindowDiff init called


In [36]:
true=[0,0,1,1,1,0,0,0,1,1,0]
pred1=[0,0,1,1,1,0,0,0,1,1,0]
pred2=[0,0,0,1,1,0,0,0,1,1,0]
pred3=[0,0,0,1,1,1,0,0,1,1,0]


In [37]:
pen=winDiff.calc_penalty(true,pred1,3)
pen

[0:2]=[0, 0, 1] true count 1 prd count 1 diff 0 penalty 0
[1:3]=[0, 1, 1] true count 1 prd count 1 diff 0 penalty 0
[2:4]=[1, 1, 1] true count 0 prd count 0 diff 0 penalty 0
[3:5]=[1, 1, 0] true count 1 prd count 1 diff 0 penalty 0
[4:6]=[1, 0, 0] true count 1 prd count 1 diff 0 penalty 0
[5:7]=[0, 0, 0] true count 0 prd count 0 diff 0 penalty 0
[6:8]=[0, 0, 1] true count 1 prd count 1 diff 0 penalty 0
[7:9]=[0, 1, 1] true count 1 prd count 1 diff 0 penalty 0


0.0

In [38]:
pen=winDiff.calc_penalty(true,pred2,3)
pen

[0:2]=[0, 0, 1] true count 1 prd count 0 diff 1 penalty 1
[1:3]=[0, 1, 1] true count 1 prd count 1 diff 0 penalty 1
[2:4]=[1, 1, 1] true count 0 prd count 1 diff 1 penalty 2
[3:5]=[1, 1, 0] true count 1 prd count 1 diff 0 penalty 2
[4:6]=[1, 0, 0] true count 1 prd count 1 diff 0 penalty 2
[5:7]=[0, 0, 0] true count 0 prd count 0 diff 0 penalty 2
[6:8]=[0, 0, 1] true count 1 prd count 1 diff 0 penalty 2
[7:9]=[0, 1, 1] true count 1 prd count 1 diff 0 penalty 2


0.25

In [39]:
pen=winDiff.calc_penalty(true,pred3,3)
pen

[0:2]=[0, 0, 1] true count 1 prd count 0 diff 1 penalty 1
[1:3]=[0, 1, 1] true count 1 prd count 1 diff 0 penalty 1
[2:4]=[1, 1, 1] true count 0 prd count 1 diff 1 penalty 2
[3:5]=[1, 1, 0] true count 1 prd count 0 diff 1 penalty 3
[4:6]=[1, 0, 0] true count 1 prd count 1 diff 0 penalty 3
[5:7]=[0, 0, 0] true count 0 prd count 1 diff 1 penalty 4
[6:8]=[0, 0, 1] true count 1 prd count 1 diff 0 penalty 4
[7:9]=[0, 1, 1] true count 1 prd count 1 diff 0 penalty 4


0.5

In [40]:
pen=winDiff.calc_penalty(true,pred3,4)
pen

[0:3]=[0, 0, 1, 1] true count 1 prd count 1 diff 0 penalty 0
[1:4]=[0, 1, 1, 1] true count 1 prd count 1 diff 0 penalty 0
[2:5]=[1, 1, 1, 0] true count 1 prd count 1 diff 0 penalty 0
[3:6]=[1, 1, 0, 0] true count 1 prd count 1 diff 0 penalty 0
[4:7]=[1, 0, 0, 0] true count 1 prd count 1 diff 0 penalty 0
[5:8]=[0, 0, 0, 1] true count 1 prd count 2 diff 1 penalty 1
[6:9]=[0, 0, 1, 1] true count 1 prd count 1 diff 0 penalty 1


0.14285714285714285

In [43]:
tr5=[0,1,1,0,1,1,1,1,1,1,1,1,1,1,0]
pr5=[0,0,1,1,0,1,1,1,1,1,1,1,1,1,1]
pen=winDiff.calc_penalty(tr5,pr5,10)
pen

[0:9]=[0, 1, 1, 0, 1, 1, 1, 1, 1, 1] true count 3 prd count 3 diff 0 penalty 0
[1:10]=[1, 1, 0, 1, 1, 1, 1, 1, 1, 1] true count 2 prd count 3 diff 1 penalty 1
[2:11]=[1, 0, 1, 1, 1, 1, 1, 1, 1, 1] true count 2 prd count 2 diff 0 penalty 1
[3:12]=[0, 1, 1, 1, 1, 1, 1, 1, 1, 1] true count 1 prd count 2 diff 1 penalty 2
[4:13]=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1] true count 0 prd count 1 diff 1 penalty 3


0.6

In [15]:
crf_pred_db=common_utils.load_db(dir_name,crf_pred_name)
crf_pred_db

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/cv_crf_best_params.78.csv,  index False


,crf_group,crf_split,crf_predicted,crf_true,crf_proba_0,crf_proba_1
0,4,0,not_nar,not_nar,0.945664,0.054336
1,4,0,not_nar,not_nar,0.961845,0.038155
2,4,0,not_nar,not_nar,0.972418,0.027582
3,4,0,not_nar,not_nar,0.898435,0.101565
4,4,0,not_nar,not_nar,0.797448,0.202552
...,...,...,...,...,...,...
38776,79,9,not_nar,not_nar,0.969682,0.030318
38777,79,9,not_nar,not_nar,0.958210,0.041790
38778,79,9,not_nar,not_nar,0.976662,0.023338
38779,79,9,not_nar,not_nar,0.978750,0.021250


In [18]:
imp.reload(classes)
wd=classes.WindowDiff()

def calc_error(df_sub,wind_size):
    return pd.Series(wd.calc_penalty(df_sub['crf_true'],df_sub['crf_predicted'],wind_size))

wind_size=20
df=pd.DataFrame()
df[['error','len','true_count','pred_count']]=crf_pred_db.groupby(['crf_group']).apply(func=calc_error,wind_size=wind_size)
df

WindowDiff init called


,error,len,true_count,pred_count
crf_group,,,,
1,0.688027,613.0,0.0,0.0
2,0.341772,652.0,0.0,0.0
3,0.537764,682.0,0.0,0.0
4,0.297118,471.0,0.0,0.0
5,0.766734,513.0,0.0,0.0
...,...,...,...,...
76,0.129670,475.0,0.0,0.0
77,0.529833,439.0,0.0,0.0
78,0.771084,186.0,0.0,0.0


## Try new metric - WinPR

In [136]:

y_pred={}
y_pred['a']=[0,0,0,0,0,0,1,1,1,1,1,1]
y_pred['b']=[0,0,0,0,0,0,0,0,0,0,0,0]
y_pred['c']=[0,0,0,0,0,1,1,1,1,1,1,1]
y_pred['d']=[0,1,1,1,1,1,0,0,0,0,0,0]
y_pred['e']=[0,0,0,0,0,1,0,1,1,1,1,1]


In [157]:
imp.reload(classes)
wd=classes.WinPR(3)

WinPR init called


In [158]:
for k,v in y_pred.items():
    res=wd.calc_errors(y_pred['a'],v)
    print(k,res,sum([v for v in res.values()]))

a {'tp': 1.0, 'tn': 10.0, 'fp': 0.0, 'fn': 0.0} 11.0
b {'tp': 0.0, 'tn': 10.0, 'fp': 0.0, 'fn': 1.0} 11.0
c {'tp': 0.75, 'tn': 9.75, 'fp': 0.25, 'fn': 0.25} 11.0
d {'tp': 1.0, 'tn': 9.0, 'fp': 1.0, 'fn': 0.0} 11.0
e {'tp': 1.0, 'tn': 8.0, 'fp': 2.0, 'fn': 0.0} 11.0


In [170]:
imp.reload(classes)
def score_func(y, y_pred, **kwargs):
    wd=classes.WinPR(3)
    return wd.get_score(y,y_pred)

In [171]:
for k,v in y_pred.items():
    res=score_func(y_pred['a'],v)
    print(k,res)

WinPR init called
a (1.0, 1.0, 1.0)
WinPR init called
b (0, 0, 0)
WinPR init called
c (0.75, 0.75, 0.75)
WinPR init called
d (0.5, 1.0, 0.6666666666666666)
WinPR init called
e (0.3333333333333333, 1.0, 0.5)


### My Window Diff

In [10]:
true=[0,0,1,1,1,0]
pred={}
pred[0]=[0,0,0,1,1,1]

In [11]:
# np.where(true[:-1] != true[1:])[0]
np.diff(true)

array([ 0,  1,  0,  0, -1])

In [12]:
import nltk
nltk.edit_distance(true,pred[0])

2

In [14]:
cnt = np.count_nonzero(np.diff(true))
cnt

2

# SegEval

In [15]:
import segeval

In [17]:
dataset = segeval.HEARST_1997_STARGAZER

In [18]:
dataset

Dataset(dict,
        {'stargazer': {'1': (2, 3, 3, 1, 3, 6, 3),
          '2': (2, 8, 2, 4, 2, 3),
          '3': (2, 1, 2, 3, 1, 3, 1, 3, 2, 2, 1),
          '4': (2, 1, 4, 1, 1, 3, 1, 4, 3, 1),
          '5': (3, 2, 4, 3, 5, 4),
          '6': (2, 3, 4, 2, 2, 5, 3),
          '7': (2, 3, 2, 2, 3, 1, 3, 2, 3)}})

In [19]:
segmentation1 = dataset['stargazer']['1']
segmentation2 = dataset['stargazer']['2']

In [25]:
type(dataset['stargazer']['2'])

tuple

In [20]:
sum(dataset['stargazer']['1'])

21

In [21]:
sum(dataset['stargazer']['2'])

21

In [22]:
segeval.boundary_similarity(segmentation1, segmentation2)

Decimal('0.5')

In [23]:
segeval.boundary_statistics(segmentation1, segmentation2)

{'count_edits': Decimal('3.5'),
 'additions': [Addition(type=1, side='a'),
  Addition(type=1, side='a'),
  Addition(type=1, side='b')],
 'substitutions': [],
 'transpositions': [Transposition(start=8, end=9, type=1)],
 'full_misses': [1, 1, 1, 1, 1],
 'boundaries_all': 11,
 'matches': [1, 1, 1],
 'pbs': 20,
 'boundary_types': frozenset({1})}